<a href="https://colab.research.google.com/github/bernhardtandy/ProjectsMLAI/blob/main/HW4_AndyBernhardt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projects in Machine Learning and AI Homework 4**
## *RNN/LSTM/GRU for TASK and Word Embeddings*
##### **Andy Bernhardt**
##### **bernha@rpi.edu**

---
## **Task 1: RNN/LSTM/GRU for TASK**

Describe the problem, why we need sequential models to solve the problem, and choose a framework (TensorFlow keras)


---
### **Part 0: Dataset, Exploratory Data Analysis and Preprocessing**


### Setup

In [3]:
# Import required libraries for project
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.utils import shuffle

import tensorflow as tf
from keras.utils.vis_utils import plot_model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

In [ ]:
# Check for GPU: Make sure the the runtime has a GPU hardware accelerator selected
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


### Load dataset from Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Add HW4 folder to path
import sys
sys.path.append('/content/drive/MyDrive/ProjectsMLAI/HW4')

In [ ]:
# Load League of Legends games dataset
df = pd.read_csv("/content/drive/MyDrive/ProjectsMLAI/HW3/Games.csv")
df = df.rename(columns={"blueDragnoType": "blueDragonType", "redDragnoType": "redDragonType"})
pd.set_option('display.max_columns', None)

### Dataset description

Link to dataset: 

### Split data into training, validation, and testing sets

In [ ]:
df

,gameId,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueFirstBlood,blueKill,blueDeath,blueAssist,blueWardPlaced,blueWardKills,blueFirstTower,blueFirstInhibitor,blueFirstTowerLane,blueTowerKills,blueMidTowerKills,blueTopTowerKills,blueBotTowerKills,blueInhibitor,blueFirstDragon,blueDragonType,blueDragon,blueRiftHeralds,redWins,redTotalGolds,redCurrentGolds,redTotalLevel,redAvgLevel,redTotalMinionKills,redTotalJungleMinionKills,redFirstBlood,redKill,redDeath,redAssist,redWardPlaced,redWardKills,redFirstTower,redFirstInhibitor,redFirstTowerLane,redTowerKills,redMidTowerKills,redTopTowerKills,redBotTowerKills,redInhibitor,redFirstDragon,redDragonType,redDragon,redRiftHeralds
0,4247263043,0,24081,1190,44,8.8,309,74,0,8,14,14,29,8,0,0,[],0,0,0,0,0,0,[],0,0,1,30099,6073,48,9.6,366,76,0,14,8,34,31,12,1,0,['MID_LANE'],2,1,0,1,0,1,"['WATER_DRAGON', 'EARTH_DRAGON']",2,1
1,4247155821,1,24162,2212,46,9.2,393,64,0,5,6,5,31,6,0,0,[],0,0,0,0,0,0,[],0,1,0,26015,3900,48,9.6,394,89,0,6,5,5,26,11,1,0,['TOP_LANE'],1,0,1,0,0,0,[],0,0
2,4243963257,0,22413,1563,41,8.2,300,62,0,5,20,8,28,7,0,0,[],0,0,0,0,0,0,[],0,0,1,34296,5496,50,10.0,388,85,0,20,5,20,39,9,1,0,['BOT_LANE'],4,2,1,1,0,1,"['FIRE_DRAGON', 'EARTH_DRAGON']",2,1
3,4241678498,0,23837,3197,46,9.2,370,96,0,6,13,9,108,9,0,0,[],0,0,0,0,0,1,['AIR_DRAGON'],1,0,1,27824,5223,48,9.6,405,72,0,13,6,24,26,13,1,0,['TOP_LANE'],1,0,1,0,0,0,[],0,1
4,4241538868,1,27688,3663,44,8.8,381,66,0,9,10,10,36,11,0,0,[],1,0,1,0,0,1,['WATER_DRAGON'],1,1,0,25826,2909,47,9.4,329,87,0,10,9,17,41,9,1,0,['BOT_LANE'],2,1,0,1,0,0,[],0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26829,4143231833,0,26110,1535,46,9.2,289,73,0,15,14,25,38,9,0,0,[],0,0,0,0,0,0,['EARTH_DRAGON'],1,0,1,27301,3931,47,9.4,323,84,0,14,15,17,30,11,1,0,['BOT_LANE'],1,0,0,1,0,1,['WATER_DRAGON'],1,1
26830,4157911901,0,57503,3293,62,12.4,329,28,0,27,37,33,36,2,1,0,['TOP_LANE'],5,3,2,0,1,1,"['WATER_DRAGON', 'AIR_DRAGON', 'FIRE_DRAGON']",3,0,1,62919,6253,64,12.8,428,120,0,37,27,29,41,0,0,1,[],6,2,1,3,1,0,[],0,0
26831,3764171638,0,26091,2986,47,9.4,338,86,0,7,6,8,40,12,1,0,['TOP_LANE'],1,0,1,0,0,0,[],0,0,1,25579,1897,46,9.2,377,88,0,6,7,8,80,8,0,0,[],0,0,0,0,0,0,[],0,0
26832,4110201724,1,24734,4289,45,9.0,328,64,0,11,7,20,33,10,0,0,[],0,0,0,0,0,1,['FIRE_DRAGON'],1,0,0,23593,4668,43,8.6,343,67,0,7,11,11,31,13,1,0,['BOT_LANE'],1,0,0,1,0,0,['AIR_DRAGON'],1,0


In [ ]:
# Split dataframe into training (70%), validation (20%), and testing (10%) sets
df_train, df_test = train_test_split(df, test_size=0.10, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=2./9., random_state=42)
df_train.shape, df_val.shape, df_test.shape

((18783, 51), (5367, 51), (2684, 51))

### Exploratory Data Analysis

---
### **Part 1: RNN Implementation**


---
### **Part 2: LSTM and GRU Implementation**


---
### **Part 3: Considering a Feed-Forward Network for TASK**


---
## **Task 2: FastText Embeddings for Word Similarity and Dissimilarity**
 
In this section, we load pre-trained 300-dimensional FastText English word embeddings (https://fasttext.cc/docs/en/python-module.html), define functions for cosine similarity and dissimilarity of two words (using these embeddings), and show the similarity and dissimilarity scores for various pairs of words.

In [2]:
# Install and load FastText library for Python 
!pip install fasttext
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.7 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3169828 sha256=f95ff06885325a662f47aa12cc1e858d28a845d6227f132fad36ddbc2714db0f
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [4]:
# Download the pre-trained 300-dimensional FastText English word embedding model and load the model 
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')

In [33]:
# This function calculates the cosine similarity of the FastText embeddings of word_1 and word_2
# The output ranges from -1 (vectors are in opposite directions)
# to 1 (indicating maximum similarity (vectors are in the same direction))
# An output of 0 indicates that words are not related (vectors are orthogonal)
def cosine_similarity(word_1, word_2):
  x = ft.get_word_vector(word_1)
  y = ft.get_word_vector(word_2)
  return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

As dissimilarity metrics, we note that we can use vector distance metrics such as Euclidean distance, Manhattan distance, Chebyshev distance etc.... However, Euclidean distance is not good in high dimensional spaces due to increased sparsity, and Manhattan distance can also suffer. We will use 1 - cosine similarity, as, intuitively, words that are similar should be less dissimilar, and words that are not similar should be more dissimilar. If dissimilarity is the opposite of similarity, and cosine similarity is a strong measure of similarity (by vector direction, independent of magnitude), then 1 - cosine similarity should be a good measure of dissimilarity. We note that since cosine similarity ranges from -1 to 1, this dissimilarity measure ranges from 0 to 2. However, since a similarity of 0 indicates unrelatedness, we can interpret a dissimilarity of 1 as unrelated. 

In [34]:
# This function calculates the dissimilarity of the FastText embeddings of word_1 and word_2 using 1 - cosine similarity
# The output ranges from 0 (indicating no dissimilarity (vectors are in the same direction))
# to 2 (indicating maximum dissimilarity (vectors are in the opposite directions))
# An output of 1 indicates that the words are not related (vectors are orthogonal)
def dissimilarity(word_1, word_2):
  return 1 - cosine_similarity(word_1, word_2)

In [35]:
# Consider two words "sadness" and "sorrow", which we may consider similar
word_1 = "sadness"
word_2 = "sorrow"
print(f"Cosine Similarity between {word_1} and {word_2}: {cosine_similarity(word_1, word_2)}")
print(f"Dissimilarity between {word_1} and {word_2}: {dissimilarity(word_1, word_2)}")

Cosine Similarity between sadness and sorrow: 0.8628993630409241
Dissimilarity between sadness and sorrow: 0.13710063695907593


In [36]:
# Consider two words "anger" and "rage", which we may consider similar
word_1 = "anger"
word_2 = "rage"
print(f"Cosine Similarity between {word_1} and {word_2}: {cosine_similarity(word_1, word_2)}")
print(f"Dissimilarity between {word_1} and {word_2}: {dissimilarity(word_1, word_2)}")

Cosine Similarity between anger and rage: 0.7166127562522888
Dissimilarity between anger and rage: 0.2833872437477112


In [37]:
# Consider two words "the" and "microphone", which we may consider dissimilar
word_1 = "the"
word_2 = "microphone"
print(f"Cosine Similarity between {word_1} and {word_2}: {cosine_similarity(word_1, word_2)}")
print(f"Dissimilarity between {word_1} and {word_2}: {dissimilarity(word_1, word_2)}")

Cosine Similarity between the and microphone: 0.15792231261730194
Dissimilarity between the and microphone: 0.8420776873826981


In [38]:
# Consider two words "halcyon" and "paper", which we may consider dissimilar
word_1 = "halcyon"
word_2 = "paper"
print(f"Cosine Similarity between {word_1} and {word_2}: {cosine_similarity(word_1, word_2)}")
print(f"Dissimilarity between {word_1} and {word_2}: {dissimilarity(word_1, word_2)}")

Cosine Similarity between halcyon and paper: 0.026520151644945145
Dissimilarity between halcyon and paper: 0.9734798483550549
